In [ ]:
!pip install -q sentence-transformers faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 65.2 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Pequena base de textos (pode adaptar pro seu TCC)
documentos = [
    "Phishing é um tipo de golpe em que o criminoso finge ser uma instituição confiável para roubar dados sensíveis do usuário.",
    "Smishing é um phishing feito por SMS, em que mensagens falsas tentam induzir o usuário a clicar em links maliciosos.",
    "Golpes bancários digitais frequentemente envolvem falsos atendentes pedindo códigos de verificação ou senhas.",
    "Uma boa prática de segurança é nunca clicar em links recebidos por SMS de remetentes desconhecidos.",
    "Bancos legítimos não pedem senhas completas ou códigos de autenticação por telefone, e-mail ou SMS.",
    "O uso de autenticação em dois fatores ajuda a reduzir o risco de fraudes bancárias digitais.",
]


In [ ]:
# Carrega modelo de embeddings da HuggingFace
modelo = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Gera embeddings para cada documento
embeddings = modelo.encode(documentos, convert_to_numpy=True)

embeddings.shape


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(6, 384)

In [ ]:
# Dimensão dos vetores
dim = embeddings.shape[1]

# IndexFlatL2 = índice simples de similaridade por distância L2
index = faiss.IndexFlatL2(dim)

# Adiciona todos os embeddings dos documentos ao índice
index.add(embeddings)

print("Número de vetores no índice:", index.ntotal)


Número de vetores no índice: 6


In [ ]:
def buscar_documentos(pergunta: str, k: int = 3):
    """
    Recebe uma pergunta em linguagem natural,
    calcula o embedding e faz a busca semântica no índice FAISS.
    """
    # Embedding da pergunta
    q_emb = modelo.encode([pergunta], convert_to_numpy=True)

    # Busca k vizinhos mais próximos
    distancias, indices = index.search(q_emb, k)

    print(f"Pergunta: {pergunta}\n")
    print("Documentos mais relevantes:\n")

    for rank, (idx, dist) in enumerate(zip(indices[0], distancias[0]), start=1):
        print(f"{rank}. (distância={dist:.4f})")
        print(documentos[idx])
        print("-" * 80)


In [ ]:
buscar_documentos("Como me proteger de golpes por SMS?")


Pergunta: Como me proteger de golpes por SMS?

Documentos mais relevantes:

1. (distância=0.6550)
Uma boa prática de segurança é nunca clicar em links recebidos por SMS de remetentes desconhecidos.
--------------------------------------------------------------------------------
2. (distância=0.7940)
Bancos legítimos não pedem senhas completas ou códigos de autenticação por telefone, e-mail ou SMS.
--------------------------------------------------------------------------------
3. (distância=0.8318)
Smishing é um phishing feito por SMS, em que mensagens falsas tentam induzir o usuário a clicar em links maliciosos.
--------------------------------------------------------------------------------


In [ ]:
buscar_documentos("O que é smishing?")


Pergunta: O que é smishing?

Documentos mais relevantes:

1. (distância=0.9711)
Smishing é um phishing feito por SMS, em que mensagens falsas tentam induzir o usuário a clicar em links maliciosos.
--------------------------------------------------------------------------------
2. (distância=1.2878)
Phishing é um tipo de golpe em que o criminoso finge ser uma instituição confiável para roubar dados sensíveis do usuário.
--------------------------------------------------------------------------------
3. (distância=1.3247)
Bancos legítimos não pedem senhas completas ou códigos de autenticação por telefone, e-mail ou SMS.
--------------------------------------------------------------------------------


In [ ]:
buscar_documentos("Bancos pedem senha completa por telefone?")


Pergunta: Bancos pedem senha completa por telefone?

Documentos mais relevantes:

1. (distância=0.3944)
Bancos legítimos não pedem senhas completas ou códigos de autenticação por telefone, e-mail ou SMS.
--------------------------------------------------------------------------------
2. (distância=1.0308)
Uma boa prática de segurança é nunca clicar em links recebidos por SMS de remetentes desconhecidos.
--------------------------------------------------------------------------------
3. (distância=1.1373)
Golpes bancários digitais frequentemente envolvem falsos atendentes pedindo códigos de verificação ou senhas.
--------------------------------------------------------------------------------


In [ ]:
nome = "Gabriel Mantovanni"
print(f"Olá, meu nome é {nome}!")

Olá, meu nome é Gabriel Mantovanni!
